In [ ]:
import requests
from IPython.display import Image, display

In [ ]:
def acquire_access_token(TOKEN_URL, CLIENT_ID, CLIENT_SECRET):
    ''''Acquire an access token using the `client_credentials` OAuth2 flow'''
    data = {
        'grant_type': 'client_credentials', 
        'client_id': CLIENT_ID, 
        'client_secret': CLIENT_SECRET, 
        'scope': 'offline_access'};
    r = requests.post(TOKEN_URL, data=data);
    r.raise_for_status()
    access_token = r.json().get('access_token');
    return access_token

### Connection specifications

In [ ]:
# This is the token URL used for authenticating users -- DO NOT CHANGE
TOKEN_URL = "https://accounts.topio.market/auth/realms/Beta-Topio-Market/protocol/openid-connect/token"

##### From the Dashboard in the Topio account, click on menu __Clients__.

##### Press button __GENERATE__ to create a new key. You may give it a name and then click __SAVE CLIENT__.

##### Once the key is generated, your *CLIENT_SECRET* will be shown in a popup menu __only once__. Copy this value and keep it for establishing connections to your OGC services.

##### The *CLIENT_ID* of the new key will be shown in the list next to its name. 

##### This pair of values will be available for the next 24 hours. You may remove and create keys according to your needs.

In [ ]:
CLIENT_ID = "..."
# e.g., CLIENT_ID = "ccfed3f4-dd28-4ff8-aabd-1596cbd24131"

CLIENT_SECRET = "..."
# e.g., CLIENT_SECRET = "705e9d90-0f77-46c5-a9d5-53035ff275ee"

### Specify Service endpoint & Layer

##### In the Dashboard, pick menu __t OGC__ and from the list pick the WFS you want to access.
##### The URI of this WFS provides information about (1) the *service endpoint*; (2) the *service parameters*; and (3) the *layer name*. Use this information to populate the values next, as in the examples.

In [ ]:
# Specify the service endpoint URI for a WFS
WFS_ENDPOINT = "..."
# e.g.: WFS_ENDPOINT = "https://s2.ows.topio.market/p_8b3f3ee9-2065-4cff-b46f-77eac5362633/wfs"

In [ ]:
# Service parameters
WFS_COMMON_PARAMS = {
    'service': 'WFS', 
    'version': '2.0.0'
}

In [ ]:
# Specify the layer name for this OGC service
LAYER_NAME = "..."
# e.g., LAYER_NAME="_549050d0-7da5-4ace-acc7-9f4245cbe652"

### Get token from OGC server

In [ ]:
if __name__ == '__main__':
    # Acquire an access token to pass as authorization header along with our requests to an OGC service
    access_token = acquire_access_token(TOKEN_URL, CLIENT_ID, CLIENT_SECRET)
    #print(access_token)
    auth_headers = {'Authorization': f"Bearer {access_token}"}

### Execute requests against a WFS

##### Submit a *GetCapabilities* request against this WFS

In [ ]:
if __name__ == '__main__':
    r1 = requests.get(WFS_ENDPOINT, 
        params={**WFS_COMMON_PARAMS, 'request': 'GetCapabilities'}, 
        headers={**auth_headers})
    r1.raise_for_status()
    print(r1.text)

##### Submit a *DescribeFeatureType* of the given layer against this WFS

In [ ]:
if __name__ == '__main__':
    r2 = requests.get(WFS_ENDPOINT, 
        params={**WFS_COMMON_PARAMS, 'request': 'DescribeFeatureType', 'typeNames': LAYER_NAME}, 
        headers={**auth_headers})
    r2.raise_for_status()
    print(r2.text)

##### Submit a *GetFeatures* request against this WMS using a bounding box (BBOX) and get its response in JSON

In [ ]:
if __name__ == '__main__':
    # Specify minx, miny, max, maxy values for the BBOX
    # Include spacification of CRS in the bounding box in case that original data is in another CRS (Original data is in EPSG:2100)
    bbox2 = "10.80,26.92,30.74,51.92,EPSG:4326"
    r4 = requests.get(WFS_ENDPOINT, 
        params={**WFS_COMMON_PARAMS, 
                'request': 'GetFeature', 
                'outputFormat': 'json',
                'typenames': LAYER_NAME,
                'srsName': 'EPSG:4326',
                'bbox': bbox2},
                headers={**auth_headers})
    r4.raise_for_status()
    # Response is in JSON
    print("GetFeature responded with type: " + r4.headers['Content-Type'])
    
    # Save response to a JSON file
    filename4 = f"getfeature-{LAYER_NAME}.json"
    print(f"Saving response to {filename4}")
    with open(filename4, 'wb') as f: 
        f.write(r4.content);